# Warning:

This service is largely unused, and will be rescoped in the future. As such, both the service and this test suite are somewhat outdated and brittle. These tests should be tidied up as changes occur to HGA to reflect the new scope and capabilities of the service. Initial changes will likely include the removal of browse image generation and netCDF4 handling.

# Harmony GDAL Adapter (HGA) regression tests

The scope of this Jupyter notebook is to run a suite of regression tests against sample collections for the Harmony GDAL Adapter (HGA). These sample collections are primarily from the Alaska Satellite Facility (ASF), including:

* [ALOS AVNIR OBS ORI](https://cmr.earthdata.nasa.gov/search/concepts/C1808440897-ASF.html)
* [UAVSAR POLSAR Pauli](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1244141264-EEDTEST.html)

**NOTE:**

Previously, this notebook also contained tests for the [Sentinel 1 Interferograms collection](https://cmr.earthdata.nasa.gov/search/concepts/C1595422627-ASF.html). These tests were removed as HGA alone no longer supports netCDF4 input files (or netCDF4 inputs contained in a zip file). If a future service chain is created with net2cog as a preceding step to HGA, then those tests could represent a good baseline for a regression test suite of that service chain. Those tests can be found in [this version of the notebook](https://github.com/nasa/harmony-regression-tests/blob/68fd8b684d22937933260825f1738aa2f9ed9600/test/hga/HGA_Regression.ipynb).

## Set the Harmony environment:

The cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

The default value is for the UAT environment. When using this notebook there are two ways to use the non-default environment:

* Run this notebook in a local Jupyter notebook server and change the value of `harmony_host_url` in the cell below to the value for the environment you require from the above list.
* Use the `run_notebooks.sh` script, which requires you to declare an environment variable `HARMONY_HOST_URL`. Set that environment variable to the value above that corresponds to the environment you want to test. That environment variable will take precedence over the default value in the cell below.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-hga`

A `.netrc` file must also be located in the `test` directory of this repository.

### Import required packages:

In [ ]:
from datetime import datetime
from os.path import exists

from earthdata_hashdiff import (
    geotiff_matches_reference_hash_file,
    nc4_matches_reference_hash_file,
)
from harmony import BBox, Client, Collection, Environment, Request

### Import shared utilities:

In [ ]:
import sys

sys.path.append('../shared_utils')
from utilities import print_success, submit_and_download

### Set up environment dependent variables:

This includes the Harmony `Client` object and `Collection` objects for each of the collections for which there are regression tests. The local, SIT and UAT Harmony instances all utilise resources from CMR UAT, meaning any non-production environment will use the same resources.

When adding a production entry to the dictionary below, the collection instances can be included directly in the production dictionary entry, as they do not need to be shared.

In [ ]:
non_production_collections = {
    'avnir_collection': Collection(id='C1244141281-EEDTEST'),
    'sentinel_collection': Collection(id='C1244141250-EEDTEST'),
    'uavsar_collection': Collection(id='C1244141264-EEDTEST'),
    'mur_collection': Collection(id='C1238621141-POCLOUD'),
}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        'env': Environment.UAT,
        **non_production_collections,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        'env': Environment.SIT,
        **non_production_collections,
    },
    'http://localhost:3000': {'env': Environment.LOCAL, **non_production_collections},
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])

### Define helper function:

In [ ]:
def get_test_granule_id(
    environment: Environment,
    production_granule_id: str,
    non_prod_granule_id: str,
) -> str:
    """Select either the production or non-production granule concept ID
    based on the environment against which the tests are being run.

    """
    if environment == Environment.PROD:
        granule_id = production_granule_id
    else:
        granule_id = non_prod_granule_id

    return granule_id

## AVNIR

The tests below will use the ALOS AVNIR OBS ORI collection (for UAT this is mirrored in the EEDTEST CMR environment). There are no reference images provided for this collection, as they would each be large (tens of MB).

### AVNIR test case 1:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_request_one = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-one'],
    )

    avnir_output_one_file_name = 'avnir_results_one.tif'
    submit_and_download(harmony_client, avnir_request_one, avnir_output_one_file_name)

    assert exists(
        avnir_output_one_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box multiple variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_one_file_name,
        'reference_images/avnir_reference_one.json',
    ), 'Test output does not match reference file, AVNIR bounding box multiple variable request.'

    print_success('HGA AVNIR (zipfile input) bounding box multiple variable request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 2:

This test case combines a bounding box spatial subset across the Prime Meridian with a variable subset of bands 3 and 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1817336128-ASF', 'G1244144599-EEDTEST'
    )

    avnir_request_two = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-0.15, 14.8, 0.15, 14.9),
        variables=['Band3', 'Band4'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-two'],
    )

    avnir_output_two_file_name = 'avnir_results_two.tif'
    submit_and_download(harmony_client, avnir_request_two, avnir_output_two_file_name)

    assert exists(
        avnir_output_two_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (Prime Meridian) multiple variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_two_file_name,
        'reference_images/avnir_reference_two.json',
    ), 'Test output does not match reference file, AVNIR bounding box (Prime Meridian) multiple variable request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (Prime Meridian) multiple variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 3:

This test combines a bounding box spatial subset across the anti-meridian with a variable subset for bands 1 and 2.  

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1835935580-ASF', 'G1244144598-EEDTEST'
    )

    avnir_directory_three = 'avnir_test_three'

    avnir_request_three = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(179.75, 66.5, -179.85, 66.75),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-three'],
    )

    avnir_output_three_file_name = 'avnir_results_three.tif'
    submit_and_download(
        harmony_client, avnir_request_three, avnir_output_three_file_name
    )

    assert exists(
        avnir_output_three_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (antimeridian) multiple variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_three_file_name,
        'reference_images/avnir_reference_three.json',
    ), 'Test output does not match reference file, AVNIR bounding box (antimeridian) multiple variable request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (antimeridian) multiple variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 4:

This test case combines a bounding box spatial subset in the Northern Hemisphere with a variable subset requesting only band 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 1 output band.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1849288272-ASF', 'G1244144595-EEDTEST'
    )

    avnir_request_four = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-99.15, 36.9, -98.85, 37.1),
        variables=['Band4'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-four'],
    )

    avnir_output_four_file_name = 'avnir_results_four.tif'
    submit_and_download(harmony_client, avnir_request_four, avnir_output_four_file_name)

    assert exists(
        avnir_output_four_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (Northern Hemisphere) single variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_four_file_name,
        'reference_images/avnir_reference_four.json',
    ), 'Test output does not match reference file, AVNIR bounding box (Northern Hemisphere) single variable request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (Northern Hemisphere) single variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 5:

This test case requests a bounding box spatial subsest in the Northern Hemisphere. No variable subset is specified, so the request will retrieve all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all variables and expect 4 output variables.
This output is identical to the output if each band is requested e.g. adding `variables=['Band1','Band2', 'Band3', 'Band4']` to the Request.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1849288272-ASF', 'G1244144595-EEDTEST'
    )

    avnir_request_five = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-99.2, 36.95, -98.9, 37.05),
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-five'],
    )

    avnir_output_five_file_name = 'avnir_results_five.tif'
    submit_and_download(harmony_client, avnir_request_five, avnir_output_five_file_name)

    assert exists(
        avnir_output_five_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (Northern Hemisphere) all-variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_five_file_name,
        'reference_images/avnir_reference_five.json',
    ), 'Test output does not match reference file, AVNIR bounding box (Northern Hemisphere) all-variable request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (Northern Hemisphere) all-variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 6:

This test combines a bounding box spatial subset in the Southern Hemisphere with a variable subset requesting bands 1 and 3 and specifies the height and width of the output bands.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1835549401-ASF', 'G1244144601-EEDTEST'
    )

    avnir_request_six = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(18.07, -26.1, 18.25, -25.8),
        variables=['Band1', 'Band3'],
        format='image/tiff',
        granule_id=granule_id,
        height=30,
        width=75,
        labels=['hga-rtest', 'hga-rtest-avnir-six'],
    )

    avnir_output_six_file_name = 'avnir_results_six.tif'
    submit_and_download(harmony_client, avnir_request_six, avnir_output_six_file_name)

    assert exists(
        avnir_output_six_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (Southern Hemisphere) multiple variable request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_six_file_name,
        'reference_images/avnir_reference_six.json',
    ), 'Test output does not match reference file, AVNIR bounding box (Southern Hemisphere) multiple variable request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (Southern Hemisphere) multiple variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 7:

This test combines a bounding box spatial subset (with negative longitude values) with all bands being explicitly specified alongside the height and width of the output array.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1906378530-ASF', 'G1244144603-EEDTEST'
    )

    avnir_request_seven = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-97.65, 49.65, -97.35, 49.95),
        variables=['Band1', 'Band2', 'Band3', 'Band4'],
        granule_id=granule_id,
        height=45,
        width=90,
        format='image/tiff',
        labels=['hga-rtest', 'hga-rtest-avnir-seven'],
    )

    avnir_output_seven_file_name = 'avnir_results_seven.tif'
    submit_and_download(
        harmony_client, avnir_request_seven, avnir_output_seven_file_name
    )

    assert exists(
        avnir_output_seven_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box (negative longitudes) with image dimensions request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_seven_file_name,
        'reference_images/avnir_reference_seven.json',
    ), 'Test output does not match reference file, AVNIR bounding box (negative longitudes) with image dimensions request.'

    print_success(
        'HGA AVNIR (zipfile input) bounding box (negative longitudes) with image dimensions request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 8:

This test requests a bounding box spatial subset, where the bounding box is larger than granule itself. No variables are specified, meaning the request should return all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813329318-ASF', 'G1244144596-EEDTEST'
    )

    avnir_request_eight = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-79.9, 9.2, -78.9, 10.3),
        granule_id=granule_id,
        format='image/tiff',
        labels=['hga-rtest', 'hga-rtest-avnir-eight'],
    )

    avnir_output_eight_file_name = 'avnir_results_eight.tif'
    submit_and_download(
        harmony_client, avnir_request_eight, avnir_output_eight_file_name
    )

    assert exists(
        avnir_output_eight_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) large bounding box request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_eight_file_name,
        'reference_images/avnir_reference_eight.json',
    ), 'Test output does not match reference file, AVNIR large bounding box request.'

    print_success('HGA AVNIR (zipfile input) large bounding box request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 9:

This test combines a bounding box spatial subset with a temporal range subset. All variables are explicitly specified. No granules are specified in this request, meaning the results will be determined by those granules that match the spatial and temporal criteria.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are requesting "all" variables and expecting to retrieve all 4 output bands.


In [ ]:
if environment_information is not None:
    avnir_request_nine = Request(
        collection=environment_information['avnir_collection'],
        variables=['all'],
        spatial=BBox(18.4, -26.1, 18.6, -26.0),
        format='image/tiff',
        labels=['hga-rtest', 'hga-rtest-avnir-nine'],
    )

    avnir_output_nine_file_name = 'avnir_results_nine.tif'
    submit_and_download(harmony_client, avnir_request_nine, avnir_output_nine_file_name)

    assert exists(
        avnir_output_nine_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) bounding box and temporal subset request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_nine_file_name,
        'reference_images/avnir_reference_nine.json',
    ), 'Test output does not match reference file, AVNIR bounding box and temporal subset request.'

    print_success('HGA AVNIR (zipfile input) bounding box and temporal subset request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 10:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. No format is specified, such that the default of a GeoTIFF output should be returned.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_request_ten = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        variables=['Band1', 'Band2'],
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-ten'],
    )

    avnir_output_ten_file_name = 'avnir_results_ten.tif'
    submit_and_download(harmony_client, avnir_request_ten, avnir_output_ten_file_name)

    assert exists(
        avnir_output_ten_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) default format request.'

    assert geotiff_matches_reference_hash_file(
        avnir_output_ten_file_name,
        'reference_images/avnir_reference_ten.json',
    ), 'Test output does not match reference file, AVNIR default format request.'

    print_success('HGA AVNIR (zipfile input) default format request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 11:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. A netCDF4 output format is specified.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

**Note: earthdata-hashdiff comparison is disabled because it requires Python 3.11, and that is incompatible with the version of GDAL used for GeoTIFF comparisons. This test is retained to ensure the request runs without error.**

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_request_eleven = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        format='application/x-netcdf4',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-avnir-eleven'],
    )

    avnir_output_eleven_file_name = 'avnir_results_eleven.tif'
    submit_and_download(
        harmony_client, avnir_request_eleven, avnir_output_eleven_file_name
    )

    assert exists(
        avnir_output_eleven_file_name,
    ), 'Unsuccessful HGA AVNIR (zipfile input) netCDF4 format request.'

    assert nc4_matches_reference_hash_file(
        avnir_output_eleven_file_name,
        'reference_images/avnir_reference_eleven.json',
    ), 'Test output does not match reference file, AVNIR netCDF4 format request.'

    print_success('HGA AVNIR (zipfile input) netCDF4 format request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

## UAVSAR POLSAR Pauli

The tests below will use the UAVSAR POLSAR Pauli collection (for UAT this is mirrored in the EEDTEST CMR environment).

The UAVSAR POLSAR data consists of 3 variables of data where each band represents a science variable (e.g., "HH minus VV, Red Band", "HV,Green Band" and "HH plus VV, Blue Band").

### UAVSAR test case 1:

This test combines a bounding box spatial subset with a variable subset (`Band1`).



In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_request_one = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-one'],
    )

    uavsar_output_one_file_name = 'uavsar_results_one.tif'
    submit_and_download(harmony_client, uavsar_request_one, uavsar_output_one_file_name)

    assert exists(
        uavsar_output_one_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box, variable 1 subset request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_one_file_name,
        'reference_images/uavsar_reference_one.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box, variable 1 subset request.'

    print_success('HGA UAVSAR (GeoTIFF input) bounding box, variable 1 subset request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 2:

This test combines a bounding box spatial subset with a variable subset (`Band2`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_request_two = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.9, 63.75, -145.8, 63.80),
        variables=['Band2'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-two'],
    )

    uavsar_output_two_file_name = 'uavsar_results_two.tif'
    submit_and_download(harmony_client, uavsar_request_two, uavsar_output_two_file_name)

    assert exists(
        uavsar_output_two_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box, variable 2 subset request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_two_file_name,
        'reference_images/uavsar_reference_two.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box, variable 2 subset request.'

    print_success('HGA UAVSAR (GeoTIFF input) bounding box, variable 2 subset request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 3:

This test combines a bounding box spatial subset with a variable subset (`Band3`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV").

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_request_three = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.85, 63.85, -145.80, 63.90),
        variables=['Band3'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-three'],
    )

    uavsar_output_three_file_name = 'uavsar_results_three.tif'
    submit_and_download(
        harmony_client, uavsar_request_three, uavsar_output_three_file_name
    )

    assert exists(
        uavsar_output_three_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box, variable 3 subset request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_three_file_name,
        'reference_images/uavsar_reference_three.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box, variable 3 subset request.'

    print_success('HGA UAVSAR (GeoTIFF input) bounding box, variable 3 subset request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 4:

This test combines a bounding box spatial subset with a variable subset for multiple variables (`Band1` and `Band2`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") this test requests 2 bands. 

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1366089385-ASF', 'G1244144737-EEDTEST'
    )

    uavsar_request_four = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-123.39, 40.05, -123.34, 40.06),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-four'],
    )

    uavsar_output_four_file_name = 'uavsar_results_four.tif'
    submit_and_download(
        harmony_client, uavsar_request_four, uavsar_output_four_file_name
    )

    assert exists(
        uavsar_output_four_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box, multiple variable subset request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_four_file_name,
        'reference_images/uavsar_reference_four.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box, multiple variable subset request.'

    print_success(
        'HGA UAVSAR (GeoTIFF input) bounding box, multiple variable subset request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 5:

This test performs a bounding box spatial subset, while requesting all variables.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., \"HH - VV\"), so the output is expected to have a total of 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1366089385-ASF', 'G1244144737-EEDTEST'
    )

    uavsar_request_five = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-123.69, 39.95, -123.59, 40.0),
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-five'],
    )

    uavsar_output_five_file_name = 'uavsar_results_five.tif'
    submit_and_download(
        harmony_client, uavsar_request_five, uavsar_output_five_file_name
    )

    assert exists(
        uavsar_output_five_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box, all variable request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_five_file_name,
        'reference_images/uavsar_reference_five.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box, all variable request.'

    print_success('HGA UAVSAR (GeoTIFF input) bounding box, all variable request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 6:

This test performs a bounding box spatial subset that crosses the Equator, while explicitly requesting all variables by identifier "all".

Each band in the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") so the output is expected to have 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1232482059-ASF', 'G1244144732-EEDTEST'
    )

    uavsar_request_six = Request(
        collection=environment_information['uavsar_collection'],
        variables=['all'],
        spatial=BBox(11.57, -0.03, 11.60, 0.02),
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-six'],
    )

    uavsar_output_six_file_name = 'uavsar_results_six.tif'
    submit_and_download(harmony_client, uavsar_request_six, uavsar_output_six_file_name)

    assert exists(
        uavsar_output_six_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box (Equator crossing) request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_six_file_name,
        'reference_images/uavsar_reference_six.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box (Equator crossing) request.'

    print_success('HGA UAVSAR (GeoTIFF input) bounding box (Equator crossing) request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 7:

This test performs a bounding box spatial subset in the Northern Hemisphere, explicitly requesting a bounding box that is larger than the input granule extent. To limit the size of the reference image stored in the repository, the request specifies a variable subset for only `Band1`, and the output dimensions are specified to be coarser resolution than the input granule.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1402334529-ASF', 'G1244144730-EEDTEST'
    )

    uavsar_request_seven = Request(
        collection=environment_information['uavsar_collection'],
        variables=['Band1'],
        spatial=BBox(-139, 61, -137, 62),
        format='image/tiff',
        height=870,
        width=1129,
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-seven'],
    )

    uavsar_output_seven_file_name = 'uavsar_results_seven.tif'
    submit_and_download(
        harmony_client, uavsar_request_seven, uavsar_output_seven_file_name
    )

    assert exists(
        uavsar_output_seven_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) large bounding box single variable request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_seven_file_name,
        'reference_images/uavsar_reference_seven.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) large bounding box single variable request.'

    print_success(
        'HGA UAVSAR (GeoTIFF input) large bounding box single variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 8:

This test combines a bounding box spatial subset with a variable subset (`Band2`). The bounding box extends past the western bound of the input granule extent.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1223713114-ASF', 'G1244144731-EEDTEST'
    )

    uavsar_request_eight = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-72.3, -40.02, -72.0, -40.0),
        variables=['Band2'],
        format='image/tiff',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-eight'],
    )

    uavsar_output_eight_file_name = 'uavsar_results_eight.tif'
    submit_and_download(
        harmony_client, uavsar_request_eight, uavsar_output_eight_file_name
    )

    assert exists(
        uavsar_output_eight_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) overly-Western bounding box single variable request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_eight_file_name,
        'reference_images/uavsar_reference_eight.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) overly-Western bounding box single variable request.'

    print_success(
        'HGA UAVSAR (GeoTIFF input) overly-Western bounding box single variable request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 9:

This test combines a bounding box spatial subset with a temporal subset. The temporal subset should return only a single granule, and apply the spatial and variable subset to that granule.

In [ ]:
if environment_information is not None:
    uavsar_request_nine = Request(
        collection=environment_information['uavsar_collection'],
        variables=['all'],
        spatial=BBox(-89.02, 28.98, -89.00, 29.00),
        temporal={
            'start': datetime(2016, 11, 17, 23, 27, 0),
            'stop': datetime(2016, 11, 17, 23, 28, 0),
        },
        format='image/tiff',
        labels=['hga-rtest', 'hga-rtest-uavsar-nine'],
    )

    uavsar_output_nine_file_name = 'uavsar_results_nine.tif'
    submit_and_download(
        harmony_client, uavsar_request_nine, uavsar_output_nine_file_name
    )

    assert exists(
        uavsar_output_nine_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) bounding box and temporal subset request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_nine_file_name,
        'reference_images/uavsar_reference_nine.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) bounding box and temporal subset request.'

    print_success(
        'HGA UAVSAR (GeoTIFF input) bounding box and temporal subset request.'
    )
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 10:

This test combines a bounding box spatial subset with a variable subset (`Band1`). No format is specified, so the output should be a GeoTIFF.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_ten = 'uavsar_test_ten'

    uavsar_request_ten = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-ten'],
    )

    uavsar_output_ten_file_name = 'uavsar_results_ten.tif'
    submit_and_download(harmony_client, uavsar_request_ten, uavsar_output_ten_file_name)

    assert exists(
        uavsar_output_ten_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) default format request.'

    assert geotiff_matches_reference_hash_file(
        uavsar_output_ten_file_name,
        'reference_images/uavsar_reference_ten.json',
    ), 'Test output does not match reference file, HGA UAVSAR (GeoTIFF input) default format request.'

    print_success('HGA UAVSAR (GeoTIFF input) default format request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 11:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. A netCDF4 output format is specified.

**Note: earthdata-hashdiff comparison is disabled because it requires Python 3.11, and that is incompatible with the version of GDAL used for GeoTIFF comparisons. This test is retained to ensure the request runs without error.**

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_request_eleven = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        format='application/x-netcdf4',
        granule_id=granule_id,
        labels=['hga-rtest', 'hga-rtest-uavsar-eleven'],
    )

    uavsar_output_eleven_file_name = 'uavsar_results_eleven.nc'
    submit_and_download(
        harmony_client, uavsar_request_eleven, uavsar_output_eleven_file_name
    )

    assert exists(
        uavsar_output_eleven_file_name,
    ), 'Unsuccessful HGA UAVSAR (GeoTIFF input) netCDF4 format request.'

    assert nc4_matches_reference_hash_file(
        uavsar_output_eleven_file_name, 'reference_images/uavsar_reference_eleven.json'
    ), 'Test output does not match reference file, UAVSAR netCDF4 output'

    print_success('HGA UAVSAR (GeoTIFF input) netCDF4 format request.')
else:
    print('Skipping test: HGA regression tests not configured for this environment.')